<a href="https://colab.research.google.com/github/Ajayjoseph842/AI-OHS/blob/main/colab_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 STEP 1: Install libraries
!pip install --upgrade gspread pandas openpyxl gspread_dataframe requests

# 📌 STEP 2: Authenticate and connect to Google Sheets
import gspread
from google.colab import auth
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd
import requests
import time

# Authenticate with Google
auth.authenticate_user()
from google.auth import default
from google.auth.transport.requests import Request
import gspread

creds, _ = default()
creds.refresh(Request())
gc = gspread.authorize(creds)

# Connect to the spreadsheet by ID
spreadsheet = gc.open_by_key("1HwD1ZmOprV_RP1o3lgieCB6hI3R86v3PcvFciU3arxU")
worksheet = spreadsheet.sheet1  # Assuming form responses are in the first sheet

# 📌 STEP 3: Read data from the sheet
df = get_as_dataframe(worksheet).dropna(how='all')

# Rename for consistency if needed
df.columns = [c.strip() for c in df.columns]
incident_col = [col for col in df.columns if "description" in col.lower()][0]

# 📌 STEP 4: Add AI columns if not present
if 'AI Summary' not in df.columns:
    df['AI Summary'] = ""
if 'Hazard Tags' not in df.columns:
    df['Hazard Tags'] = ""

# 📌 STEP 5: OpenRouter setup
api_key = "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"  # 🔁 Replace this with your OpenRouter key
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

def summarize_incident(text):
    try:
        payload = {
            "model": "mistralai/mistral-7b-instruct",
            "messages": [
                {
                    "role": "system",
                    "content": "You are a safety officer. Summarize incident reports and tag likely hazards."
                },
                {
                    "role": "user",
                    "content": f"Incident:\n{text}\n\nProvide a short summary and likely hazard categories."
                }
            ]
        }

        res = requests.post("https://openrouter.ai/api/v1/chat/completions", json=payload, headers=headers)
        result = res.json()
        return result['choices'][0]['message']['content']

    except Exception as e:
        return f"Error: {str(e)}"

# 📌 STEP 6: Apply AI to unprocessed rows
for i, row in df.iterrows():
    if pd.isna(row['AI Summary']) or row['AI Summary'].strip() == "":
        print(f"Processing row {i+1}...")
        result = summarize_incident(row[incident_col])
        parts = result.split("\n")
        df.at[i, 'AI Summary'] = parts[0] if parts else result
        df.at[i, 'Hazard Tags'] = ", ".join(parts[1:]) if len(parts) > 1 else "Unknown"
        time.sleep(2)

# 📌 STEP 7: Export for Power BI (optional)
df.to_csv("enhanced_incidents.csv", index=False)
print("✅ Done! CSV saved.")

# 📌 STEP 8 (Optional): Push updated data back to Google Sheet
# Uncomment below line if you want to overwrite the sheet with AI results:
# set_with_dataframe(worksheet, df)
